# feature analysis result - already exist in the previous result file

We use WL or CCA to do the
1. inter-subject gesture classification
2. Few-shot adaptation

WL로 가자

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys, random
sys.path.append('../../Shared')
import processing, Model, Visualization, modules, Adaptation
from sklearn.model_selection import train_test_split



def import_data(subject, current_rep, db_name, feat_idx):
    path = f'C:/Users/hml76/Desktop/UMN/Research/5th Federated Prototype Learning/Dataset/{db_name}/'

    x_train = np.array(pd.read_csv(path + 'Sub{}/Sub{}_Rep{}_data.csv'.format(subject, subject, current_rep)))
    y_train = np.array(pd.read_csv(path + 'Sub{}/Sub{}_Rep{}_label.csv'.format(subject, subject, current_rep)))
    indices = np.random.permutation(len(x_train)) #shuffle
    x_train, y_train = x_train[indices], y_train[indices]

    x_train = x_train.reshape(len(x_train), 10, 11)
    x_train = x_train[:, :, feat_idx]
    x_train = x_train.reshape(len(x_train), 10, 1)

    return x_train, y_train

def ready_data_per_subject_all_reps(subject, Data, downsample, downsample_size, feat_idx):
    X, y = [], []
    for rep in range(1,11):
        X_tmp, y_tmp = import_data(subject, rep, Data, feat_idx)

        X.append(X_tmp)
        y.append(y_tmp)
    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    y = np.array([np.argmax(y[i]) for i in range(len(y))])

    if downsample:
        idx = np.random.choice(X.shape[0], size=downsample_size, replace=False)
        X, y = X[idx], y[idx]

    return X, y

In [ ]:
#sub_lst = random.sample(range(1, 28), 10)
sub_lst = [20, 14, 24, 9, 8, 11, 25, 17, 27, 13]

# Baseline - LOSO

Random 10 subjects

In [4]:
Data_name = "DB1"
Sub_all = []

for train_subject in sub_lst:
    print(f"Testing subject {train_subject}")
    X_train, y_train = ready_data_per_subject_all_reps(subject=train_subject, Data=Data_name, downsample=False, downsample_size=50000, feat_idx=1)

    model = Model.build_model_1D(input_shape=X_train.shape[1:], num_classes=len(np.unique(y_train)))
    history = model.fit(X_train, y_train, epochs=50, batch_size=256, verbose=0)
    Sub_acc = []
    for test_subject in sub_lst:
        X_test, y_test = ready_data_per_subject_all_reps(subject=train_subject, Data=Data_name, downsample=True, downsample_size=10000, feat_idx=1)
        loss, acc = model.evaluate(X_test, y_test, verbose=0)
        print(f"\t Testing subject {test_subject} : acc {acc*100:.2f}%")
        Sub_acc.append(acc)
    Sub_all.append(Sub_acc)

Testing subject 20
	 Testing subject 20 : acc 43.32%
	 Testing subject 14 : acc 43.19%
	 Testing subject 24 : acc 43.06%
	 Testing subject 9 : acc 43.55%
	 Testing subject 8 : acc 43.78%
	 Testing subject 11 : acc 42.85%
	 Testing subject 25 : acc 42.90%
	 Testing subject 17 : acc 43.26%
	 Testing subject 27 : acc 43.05%
	 Testing subject 13 : acc 42.72%
Testing subject 14
	 Testing subject 20 : acc 38.32%
	 Testing subject 14 : acc 38.59%
	 Testing subject 24 : acc 39.42%
	 Testing subject 9 : acc 39.42%
	 Testing subject 8 : acc 38.86%
	 Testing subject 11 : acc 38.57%
	 Testing subject 25 : acc 39.20%
	 Testing subject 17 : acc 38.96%
	 Testing subject 27 : acc 39.44%
	 Testing subject 13 : acc 39.29%
Testing subject 24
	 Testing subject 20 : acc 50.41%
	 Testing subject 14 : acc 50.03%
	 Testing subject 24 : acc 50.51%
	 Testing subject 9 : acc 50.11%
	 Testing subject 8 : acc 50.55%
	 Testing subject 11 : acc 51.17%
	 Testing subject 25 : acc 49.55%
	 Testing subject 17 : acc 50.5

In [5]:
Sub_all

[[0.43320000171661377,
  0.4318999946117401,
  0.43059998750686646,
  0.43549999594688416,
  0.43779999017715454,
  0.4284999966621399,
  0.42899999022483826,
  0.4325999915599823,
  0.43050000071525574,
  0.42719998955726624],
 [0.3831999897956848,
  0.38589999079704285,
  0.39419999718666077,
  0.39419999718666077,
  0.3885999917984009,
  0.385699987411499,
  0.3919999897480011,
  0.38960000872612,
  0.3944000005722046,
  0.3928999900817871],
 [0.5041000247001648,
  0.5002999901771545,
  0.5051000118255615,
  0.5011000037193298,
  0.5055000185966492,
  0.5116999745368958,
  0.49549999833106995,
  0.5055999755859375,
  0.5098000168800354,
  0.5070000290870667],
 [0.4097000062465668,
  0.4171000123023987,
  0.4153999984264374,
  0.4133000075817108,
  0.4138000011444092,
  0.4154999852180481,
  0.4178999960422516,
  0.41929998993873596,
  0.4129999876022339,
  0.4203000068664551],
 [0.4693000018596649,
  0.46950000524520874,
  0.47200000286102295,
  0.4668999910354614,
  0.4745999872684

# Inter-subject incremental

In [ ]:
Data_name = "DB1"
Sub_all = []

for train_subject in sub_lst:
    print(f"Testing subject {train_subject}")
    X_train, y_train = ready_data_per_subject_all_reps(subject=train_subject, Data=Data_name, downsample=False, downsample_size=50000, feat_idx=1)

    model = Model.build_model_1D(input_shape=X_train.shape[1:], num_classes=len(np.unique(y_train)))
    model.fit(X_train, y_train, epochs=50, batch_size=256, verbose=0)
    Sub_acc = []
    for test_subject in sub_lst:
        X_test, y_test = ready_data_per_subject_all_reps(subject=train_subject, Data=Data_name, downsample=True, downsample_size=10000, feat_idx=1)
        loss, acc = model.evaluate(X_test, y_test, verbose=0)
        print(f"\t Testing subject {test_subject} : acc {acc*100:.2f}%")
        Sub_acc.append(acc)
    Sub_all.append(Sub_acc)

# Few-shot adaptation